## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nenjiang,CN,2022-04-25 21:15:13,49.1833,125.2167,36.12,97,100,16.82,overcast clouds
1,1,Zyryanka,RU,2022-04-25 21:15:13,65.7500,150.8500,11.21,93,92,10.76,overcast clouds
2,2,Omsukchan,RU,2022-04-25 21:15:14,62.5333,155.8000,20.61,94,100,10.20,snow
3,3,Rikitea,PF,2022-04-25 21:15:14,-23.1203,-134.9692,76.78,74,73,14.85,light rain
4,4,Tuktoyaktuk,CA,2022-04-25 21:05:36,69.4541,-133.0374,32.95,95,100,5.93,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,2022-04-25 21:15:14,-23.1203,-134.9692,76.78,74,73,14.85,light rain
11,11,Puerto Ayora,EC,2022-04-25 21:15:17,-0.7393,-90.3518,76.96,78,86,11.14,overcast clouds
12,12,Butaritari,KI,2022-04-25 21:15:18,3.0707,172.7902,79.81,83,35,13.56,light rain
15,15,Broome,US,2022-04-25 21:15:19,42.2506,-75.8330,72.68,51,0,18.41,clear sky
17,17,Bumba,CD,2022-04-25 21:15:20,2.1833,22.4667,73.40,88,69,0.51,light rain
21,21,Cabo San Lucas,MX,2022-04-25 21:07:06,22.8909,-109.9124,84.61,60,0,10.36,clear sky
22,22,Bonthe,SL,2022-04-25 21:12:50,7.5264,-12.5050,80.69,85,83,10.98,broken clouds
23,23,Bambanglipuro,ID,2022-04-25 21:07:02,-7.9500,110.2833,76.82,89,84,3.74,broken clouds
24,24,Dingle,PH,2022-04-25 21:15:22,10.9995,122.6711,77.74,88,65,6.89,broken clouds
29,29,San Cristobal,VE,2022-04-25 21:15:23,7.7669,-72.2250,71.78,55,57,3.71,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                258
City                   258
Country                257
Date                   258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,76.78,light rain,-23.1203,-134.9692,
11,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
12,Butaritari,KI,79.81,light rain,3.0707,172.7902,
15,Broome,US,72.68,clear sky,42.2506,-75.8330,
17,Bumba,CD,73.40,light rain,2.1833,22.4667,
21,Cabo San Lucas,MX,84.61,clear sky,22.8909,-109.9124,
22,Bonthe,SL,80.69,broken clouds,7.5264,-12.5050,
23,Bambanglipuro,ID,76.82,broken clouds,-7.9500,110.2833,
24,Dingle,PH,77.74,broken clouds,10.9995,122.6711,
29,San Cristobal,VE,71.78,broken clouds,7.7669,-72.2250,


In [16]:
# Count total cities to see if matches with new count
hotel_df.count()

City                   257
Country                257
Max Temp               257
Current Description    257
Lat                    257
Lng                    257
Hotel Name             257
dtype: int64

In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
